In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import talib
from sklearn.decomposition import PCA
import math
import plotly.express as px
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split



In [4]:

PATH = "../Data/DAX/yahoo_fin/"

def load(filename="1d"):
     data = pd.read_csv(PATH+"{}.csv".format(filename))
     return data
def save(filename, data):
     data.to_csv(PATH+filename+".csv")

def normalize(x):
    mean = np.mean(x, axis=0, keepdims=True)
    std = np.std(x, axis=0, keepdims=True)
    return (x - mean) / std


In [5]:
data = load("1997-01-01-2020-07-31:1d")[33:]
print(data.columns)


Index(['Unnamed: 0', 'open', 'close', 'high', 'low', 'intra-diff',
       'inter-diff', 'intra-derivative', 'inter-derivative', '1-rad', '1-err',
       'BB-upper', 'BB-middle', 'BB-lower', 'bb-upper', 'bb-middle',
       'bb-lower', 'ema5', 'ema20', 'ema-cross', 'sma50', 'sma200',
       'sma-cross', 'sar', 'sar-diff', 'adx', 'macd', 'macdsignal', 'macdhist',
       'mom', 'rsi', 'stoch_k', 'stoch_d', 'stoch-diff', 'willr'],
      dtype='object')


In [14]:
def tf_model(period):
    feature_list = ["1-rad", "inter-derivative", 
                    "bb-upper", "bb-lower", "bb-middle", 
                    "ema-cross", "macdsignal", "macdhist", "macd", 
                    "rsi", "sar-diff", "stoch-diff", 
                    "intra-derivative", "1-err", "intra-diff"]
    
    N = len(feature_list)
    features = period[feature_list][:-1]
    features = features.values
    features = normalize(features)

    targets = period["inter-diff"][1:]
    targets = targets.values

    train_features, test_features, train_labels, test_labels = train_test_split(features, targets, test_size = 0.20, random_state = 2020)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(N),
        tf.keras.layers.Dense(128*N, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128*N, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128*N, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128*N, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # tf.keras.layers.Dense(N/2, activation='relu'),
        tf.keras.layers.Dense(1)
        ])

    model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])


    model.fit(train_features, train_labels, epochs=400, batch_size=100)
    model.evaluate(test_features,  test_labels, verbose=2)
    model.predict(test_features)


    
    

In [15]:
tf_model(data)

===============] - 2s 41ms/step - loss: 0.3676 - mean_absolute_error: 0.3676
Epoch 226/400
48/48 [==============================] - 2s 44ms/step - loss: 0.3657 - mean_absolute_error: 0.3657
Epoch 227/400
48/48 [==============================] - 2s 46ms/step - loss: 0.3647 - mean_absolute_error: 0.3647
Epoch 228/400
48/48 [==============================] - 2s 45ms/step - loss: 0.3676 - mean_absolute_error: 0.3676
Epoch 229/400
48/48 [==============================] - 2s 39ms/step - loss: 0.3580 - mean_absolute_error: 0.3580
Epoch 230/400
48/48 [==============================] - 2s 38ms/step - loss: 0.3602 - mean_absolute_error: 0.3602
Epoch 231/400
48/48 [==============================] - 2s 39ms/step - loss: 0.3589 - mean_absolute_error: 0.3589
Epoch 232/400
48/48 [==============================] - 2s 39ms/step - loss: 0.3625 - mean_absolute_error: 0.3625
Epoch 233/400
48/48 [==============================] - 2s 39ms/step - loss: 0.3554 - mean_absolute_error: 0.3554
Epoch 234/400
48/48